## Homework / Practice

### Objective: to learn more 
1. - Find out which city hosted most of matches, then sort and get the top 5 cities
2. - Find out which team won the matches and count the number of times
3. - Find out which team did make to playoff stage and how many times
4. - Find out which team made to final did not win the match
5. - Find out the match officials for Final matches
6. - Find out which venue hosted final match with year and city name
7. - Find out which are the match got tie. 

NOTE : Each time when you have new notebook, Start Spark Session and Provide datasource for Notebook

## Lead Data

In [1]:
from pyspark.sql import SparkSession

#Load File
file_path = 'Files/ipldata/ipl_summary_raw.csv'

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 3, Finished, Available, Finished)

## Find out which city hosted most of matches, then sort and get the top 5 cities

In [31]:
from pyspark.sql.functions import count

# Group by city and count the number of matches, then sort and get the top 5 cities
top_cities_df = df.groupBy("info_city").agg(count("*").alias("num_matches")).orderBy("num_matches", ascending=False).limit(5)

# Show the result
top_cities_df.show(truncate=False)



StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 33, Finished, Available, Finished)

+---------+-----------+
|info_city|num_matches|
+---------+-----------+
|Mumbai   |173        |
|Kolkata  |93         |
|Delhi    |90         |
|Chennai  |85         |
|Hyderabad|77         |
+---------+-----------+



## Find out which team won the matches and count the number of times


In [2]:
from pyspark.sql.functions import col, count

# Filter for final matches and group by the winning team, then count the number of wins
finals_wins_df = df.filter(col("info_event_stage") == "Final").groupBy("info_outcome_winner").agg(count("*").alias("num_finals_won")).orderBy("num_finals_won", ascending=False)

# Show the result
finals_wins_df.show(truncate=False)



StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 4, Finished, Available, Finished)

+---------------------+--------------+
|info_outcome_winner  |num_finals_won|
+---------------------+--------------+
|Chennai Super Kings  |5             |
|Mumbai Indians       |5             |
|Kolkata Knight Riders|3             |
|Sunrisers Hyderabad  |1             |
|Gujarat Titans       |1             |
|Deccan Chargers      |1             |
|Rajasthan Royals     |1             |
+---------------------+--------------+



## Find out which team did make to playoff stage and how many times


In [4]:
from pyspark.sql.functions import col, count

# Define the playoff stages
playoff_stages = ["Qualifier 1", "Qualifier 2", "Eliminator"]

# Filter for matches that are in any of the playoff stages and group by the first team, then count the number of appearances
playoff_appearances_team1_df = df.filter(col("info_event_stage").isin(playoff_stages)).groupBy("info_outcome_winner").agg(count("*").alias("team_made_to_payoff"))

playoff_appearances_team1_df.show()

StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 6, Finished, Available, Finished)

+--------------------+------------------+
| info_outcome_winner|num_playoffs_team1|
+--------------------+------------------+
| Sunrisers Hyderabad|                 4|
| Chennai Super Kings|                 9|
|      Gujarat Titans|                 2|
|Rising Pune Super...|                 1|
|    Rajasthan Royals|                 3|
|Royal Challengers...|                 4|
|Kolkata Knight Ri...|                 7|
|     Kings XI Punjab|                 1|
|      Delhi Capitals|                 2|
|      Mumbai Indians|                 6|
+--------------------+------------------+



## Find out which team made to final did not win the match


In [50]:
from pyspark.sql.functions import col, count

# 1. Identify teams in the final:
final_matches_df = df.filter(col("info_event_stage") == "Final")

# 2. Find teams that made it to the final but did not win:
final_teams_not_winners = final_matches_df.filter(
    (col("info_outcome_winner") == col("info_teams_1")) | (col("info_outcome_winner") == col("info_teams_2"))
)

# # 3. Extract the teams from the final matches that didn't win:
teams_not_winners = final_teams_not_winners.select(col("info_teams_1"), col("info_teams_2"), col("info_outcome_winner"),col("info_season")).distinct()
# 4. (Optional) Print the results:
print("Teams that made it to the final but did not win:")
# teams_not_winners.show(truncate=False)
teams_not_winners.show(truncate=False)

StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 52, Finished, Available, Finished)

Teams that made it to the final but did not win:
+---------------------------+---------------------------+---------------------+-----------+
|info_teams_1               |info_teams_2               |info_outcome_winner  |info_season|
+---------------------------+---------------------------+---------------------+-----------+
|Kolkata Knight Riders      |Kings XI Punjab            |Kolkata Knight Riders|2014       |
|Mumbai Indians             |Rising Pune Supergiant     |Mumbai Indians       |2017       |
|Mumbai Indians             |Chennai Super Kings        |Mumbai Indians       |2015       |
|Chennai Super Kings        |Mumbai Indians             |Chennai Super Kings  |2009/10    |
|Chennai Super Kings        |Royal Challengers Bangalore|Chennai Super Kings  |2011       |
|Gujarat Titans             |Chennai Super Kings        |Chennai Super Kings  |2023       |
|Rajasthan Royals           |Gujarat Titans             |Gujarat Titans       |2022       |
|Kolkata Knight Riders      |Ch

## Find out the match officials for Final matches


In [53]:
from pyspark.sql.functions import col

# 1. Identify teams in the final:
final_matches_df = df.filter(col("info_event_stage") == "Final")
final_match_umpires = final_matches_df.select("info_teams_1", "info_teams_2","info_officials_umpires_1","info_officials_umpires_2")
final_match_umpires.show()


StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 55, Finished, Available, Finished)

+--------------------+--------------------+------------------------+------------------------+
|        info_teams_1|        info_teams_2|info_officials_umpires_1|info_officials_umpires_2|
+--------------------+--------------------+------------------------+------------------------+
|      Mumbai Indians|Rising Pune Super...|                NJ Llong|                  S Ravi|
| Sunrisers Hyderabad| Chennai Super Kings|               M Erasmus|                  S Ravi|
|      Mumbai Indians| Chennai Super Kings|                IJ Gould|             Nitin Menon|
|      Delhi Capitals|      Mumbai Indians|             CB Gaffaney|             Nitin Menon|
| Chennai Super Kings|Kolkata Knight Ri...|             Nitin Menon|          RK Illingworth|
|    Rajasthan Royals|      Gujarat Titans|             CB Gaffaney|             Nitin Menon|
|      Gujarat Titans| Chennai Super Kings|             Nitin Menon|               RJ Tucker|
| Sunrisers Hyderabad|Kolkata Knight Ri...|           J Mada

## Find out which venue hosted final match with year and city name


In [54]:
from pyspark.sql.functions import col

# 1. Identify teams in the final:
final_matches_df = df.filter(col("info_event_stage") == "Final")
final_match_venue = final_matches_df.select("info_season", "info_city", "info_venue")
final_match_venue.show()

StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 56, Finished, Available, Finished)

+-----------+------------+--------------------+
|info_season|   info_city|          info_venue|
+-----------+------------+--------------------+
|       2017|   Hyderabad|Rajiv Gandhi Inte...|
|       2018|      Mumbai|    Wankhede Stadium|
|       2019|   Hyderabad|Rajiv Gandhi Inte...|
|    2020/21|        null|Dubai Internation...|
|       2021|       Dubai|Dubai Internation...|
|       2022|   Ahmedabad|Narendra Modi Sta...|
|       2023|   Ahmedabad|Narendra Modi Sta...|
|       2024|     Chennai|MA Chidambaram St...|
|    2007/08|      Mumbai|Dr DY Patil Sport...|
|       2009|Johannesburg|New Wanderers Sta...|
|    2009/10|      Mumbai|Dr DY Patil Sport...|
|       2011|     Chennai|MA Chidambaram St...|
|       2012|     Chennai|MA Chidambaram St...|
|       2013|     Kolkata|        Eden Gardens|
|       2014|   Bangalore|M Chinnaswamy Sta...|
|       2015|     Kolkata|        Eden Gardens|
|       2016|   Bangalore|M Chinnaswamy Sta...|
+-----------+------------+--------------

## Find out which are the match got tie. 

In [58]:
# 1. Identify teams in the final:
final_matches_df = df.filter(col("info_outcome_result") == "tie")
final_match_tie = final_matches_df.select("info_teams_1", "info_teams_2","info_season")
final_match_tie.show()


StatementMeta(, 4cd811d8-f07a-4250-9f85-56210397d923, 60, Finished, Available, Finished)

+--------------------+--------------------+-----------+
|        info_teams_1|        info_teams_2|info_season|
+--------------------+--------------------+-----------+
|       Gujarat Lions|      Mumbai Indians|       2017|
|Kolkata Knight Ri...|      Delhi Capitals|       2019|
|      Mumbai Indians| Sunrisers Hyderabad|       2019|
|      Delhi Capitals|     Kings XI Punjab|    2020/21|
|Kolkata Knight Ri...| Sunrisers Hyderabad|    2020/21|
|      Mumbai Indians|     Kings XI Punjab|    2020/21|
|Royal Challengers...|      Mumbai Indians|    2020/21|
|      Delhi Capitals| Sunrisers Hyderabad|       2021|
|Kolkata Knight Ri...|    Rajasthan Royals|       2009|
| Chennai Super Kings|     Kings XI Punjab|    2009/10|
| Sunrisers Hyderabad|Royal Challengers...|       2013|
|Royal Challengers...|    Delhi Daredevils|       2013|
|Kolkata Knight Ri...|    Rajasthan Royals|       2014|
|    Rajasthan Royals|     Kings XI Punjab|       2015|
+--------------------+--------------------+-----